# Combine results of all models:
- K-Nearest Neighbors
- Logistic Regression
- Random Forest
- Naive Bayes

**Steps:**
- Get dataframes with predictions and id
- Merge dataframes using contract id
- Compute column `PREDICTION_MEAN`
- Analyze results

In [2]:
import pandas as pd
import numpy as np

In [27]:
# Import data
contracts = pd.read_csv('../../data/csv/contract24.csv')

C:\Users\franc\AppData\Local\Temp\ipykernel_17240\2902760220.py:2: DtypeWarning: Columns (2,11) have mixed types. Specify dtype option on import or set low_memory=False.
  contracts = pd.read_csv('../../data/csv/contract24.csv')


In [5]:
contracts = contracts.drop(['Unnamed: 0', 'SF-SYSTEM', 'ANO_SID', 'ORTS-NAME', 'STRASSE', 'PARTY-ID', 'contract_year', 'Kreis'], axis=1)
contracts = contracts.dropna()

In [6]:
columns_to_encode = contracts.select_dtypes(include=['object']).columns
df_binary = pd.get_dummies(contracts, columns=columns_to_encode, drop_first=True)
df_binary.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7163047 entries, 0 to 10353243
Data columns (total 73 columns):
 #   Column                                         Dtype  
---  ------                                         -----  
 0   ORTPLZ                                         int64  
 1   SUM_INSURED                                    float64
 2   CONSTRUCTION_YEAR                              float64
 3   WFL                                            float64
 4   TYPE_OF_DEDUCTIBLE                             int64  
 5   DRAIN_PIPE_INSURED                             int64  
 6   PRIOR_DAMAGES                                  int64  
 7   UVV-KZ                                         int64  
 8   PIPE_PREMIUM_AMOUNT                            float64
 9   YEAR                                           int64  
 10  DAMAGE_FLOOD_ZONE                              float64
 11  DAMAGE_HEAVY_RAIN_ZONE                         float64
 12  LONGITUDE                                     